This file acts as a readme for the libraries created for this project.

First, we need to create a **Schema**.<br>
**Schemas** are a class containing **Types**, which are moves that can be played during the game. **Types** contain information about how they affect the score of the user who played them. Combining these creates a **Schema**, which is a ruleset for the game.

In [1]:
import rpsnetworks.schema_templates

# several schemas are predefined
RPS_SCHEMA = rpsnetworks.schema_templates.RockPaperScissors()
# observe the TYPES in this schema
RPS_SCHEMA.TYPES

{'rock': <rpsnetworks.type_templates.Rock at 0x1d6098f9f40>,
 'paper': <rpsnetworks.type_templates.Paper at 0x1d6098fa900>,
 'scissors': <rpsnetworks.type_templates.Scissors at 0x1d609a34680>}

Now we need some **Players**. Every **Player** has a **Controller** that defines how it selects moves.

In [2]:
import rpsnetworks.player_templates

# there are several templates to quickly create new players
player1 = rpsnetworks.player_templates.randomPlayer("player1", 1) # every player has an ID and starting points

# for player 2:
player_2_strategy = [0.5, 0.5, 0]
# strategies are a list of weights for choosing each move (they need not add up to 1, but it is easier to read if they do)
# indices correspond to the order of types in the schema we chose (see output of the above code block)
# this player only chooses rock or paper, 50% of the time each.

player2 = rpsnetworks.player_templates.mixedStrategyPlayer("player2", 1, player_2_strategy) # mixed players have a strategy

print(player1.CONTROLLER.ID, player2.CONTROLLER.ID)

random mixed_random


Let's run some **Games**.

In [3]:
import rpsnetworks.battle_manager

# we *always* need a manager object first
manager = rpsnetworks.battle_manager.Manager()

# it's best practice to use the runGames function to auto-manage the process of running games
rpsnetworks.battle_manager.runGames(manager, 3, # number of games to run
                                     [player1, player2], RPS_SCHEMA, {"record_players": True})

# no return value. results are stored in the manager object
manager.results

{'network_game_0': ['player2'],
 'network_game_1': ['player1'],
 'network_game_2': ['player2']}

In [4]:
# this is a little dense; it takes a bit to get used to reading
# we have 3 games here at the top level of the dict
# every turn is recorded and the moves are given in order of the indices of the players
# e.g. player2 throws rock and is the 2nd player, so the 2nd entry of the moves list that turn is rock
manager.history

{'network_game_0': [{'players_before_turn': '[{"id": "player1", "hp": 1, "controller_id": "random"}, {"id": "player2", "hp": 1, "controller_id": "mixed_random"}]',
   'players_after_turn': '[{"id": "player2", "hp": 1, "controller_id": "mixed_random"}]',
   'moves': ['scissors', 'rock'],
   'turn': 0}],
 'network_game_1': [{'players_before_turn': '[{"id": "player1", "hp": 1, "controller_id": "random"}, {"id": "player2", "hp": 1, "controller_id": "mixed_random"}]',
   'players_after_turn': '[{"id": "player1", "hp": 1, "controller_id": "random"}, {"id": "player2", "hp": 1, "controller_id": "mixed_random"}]',
   'moves': ['rock', 'rock'],
   'turn': 0},
  {'players_before_turn': '[{"id": "player1", "hp": 1, "controller_id": "random"}, {"id": "player2", "hp": 1, "controller_id": "mixed_random"}]',
   'players_after_turn': '[{"id": "player1", "hp": 1, "controller_id": "random"}]',
   'moves': ['paper', 'rock'],
   'turn': 1}],
 'network_game_2': [{'players_before_turn': '[{"id": "player1", "

Let's train a neural net now.

In [18]:
import rpsnetworks.training_manager

opponents = [player2]

train_out = rpsnetworks.training_manager.trainNetwork(
    10, 15, 20, # training parameters
    [2, 2, 3], rpsnetworks.player_templates.basicNetworkPlayer, 1, # neural network parameters
    opponents, RPS_SCHEMA # battle parameters
)

print(train_out[1], train_out[3])

<rpsnetworks.network.Network object at 0x000001D609AC5C40> {'rock': 0.0, 'paper': 1.0, 'scissors': 0.0}


Our output network always plays paper.

We can use the "verbose" flag to get basic summary data.

In [20]:
import rpsnetworks.training_manager

opponents = [
    rpsnetworks.player_templates.mixedStrategyPlayer("player1", 3, [0.5, 0, 0.5]),
    rpsnetworks.player_templates.mixedStrategyPlayer("player2", 3, [0, 0.5, 0.5])
]

train_out = rpsnetworks.training_manager.trainNetwork(
    10, 15, 20,
    [4, 2, 3], rpsnetworks.player_templates.playerAwareNetworkPlayer, 3,
    opponents, RPS_SCHEMA,
    verbose=True
)

TRAINING PARAMETERS:
Generations: 10
Children tested per generation: 15
Base # of test-games per child: 20
---------------
Schema used: Rock Paper Scissors
Opponent count: 2
---------------
RESULTS:
Network proficiency: 0.828
Move distribution:
rock: 0.673
paper: 0.000
scissors: 0.327


Documentation for the training process can be found in comments in training_manager.py.
Additional examples can be found in showcase.ipynb.